<a href="https://colab.research.google.com/github/stawiskm/QSAR_Modelbuilding_amesTest/blob/main/Genotoxicity_all_DeploymentModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Project - Computational Ames test [Part 1] Descriptor Calculation and Dataset Preparation**

Marc Jermann

In this Jupyter notebook, we will be building a real-life **data science project**. Particularly, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 1**, we will be calculating molecular descriptors that are essentially quantitative description of the compounds in the dataset. Finally, we will be preparing this into a dataset for subsequent model building in Part 2.

**with all available data**

---

## **Download PaDEL-Descriptor**

In [1]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

--2022-05-18 22:09:05--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2022-05-18 22:09:05--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M  --.-KB/s    in 0.1s    

2022-05-18 22:09:05 (165 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2022-05-18 22:09:06--  https://github

In [2]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

## **Load bioactivity data**

Download the curated ChEMBL bioactivity data that has been pre-processed from Parts 1 and 2 of this Bioinformatics Project series. Here we will be using the **bioactivity_data_3class_pIC50.csv** file that essentially contain the pIC50 values that we will be using for building a regression model.

In [3]:
import pandas as pd

In [4]:
# CHEMBL data
df1 = pd.read_csv("https://raw.githubusercontent.com/stawiskm/QSAR_Modelbuilding_amesTest/main/data/QSAR_ames_raw-data.csv")
df1 = df1.drop(labels=['molecule_chembl_id'],axis=1)
df1.columns = ['canonical_smiles', 'class']

In [5]:
df1

,canonical_smiles,class
0,CC(C)(N)C(=O)N[C@H](COCc1ccccc1)c1nnnn1CCOC(=O...,Non-toxic
1,C=C1C(=O)O[C@@H]2C[C@@]3(C)CCCC(=C)[C@@H]3C[C@...,Toxic
2,C=C1CCC[C@]2(C)C[C@H]3OC(=O)[C@@H](C)[C@H]3C[C...,Non-toxic
3,O=c1ccc2ccccc2o1,Non-toxic
4,COc1c2ccoc2cc2oc(=O)ccc12,Non-toxic
...,...,...
650,COCCCOc1cc2c(cc1OC)-c1cc(=O)c(C(=O)O)cn1[C@H](...,Non-toxic
651,N#C/C(=C\c1cccc2cnccc12)c1c[nH]c2ccccc12,Non-toxic
652,O=c1cc(NC2CCN(S(=O)(=O)C(F)(F)F)CC2)c2cc(C(c3c...,Non-toxic
653,N#Cc1cnc2ccc(-c3c(-c4ccc(F)c(Cl)c4)ncn3CCO)nn12,Non-toxic


In [6]:
df2 = pd.read_csv('https://github.com/stawiskm/QSAR_Modelbuilding_amesTest/raw/main/data/paperData1.csv')
df3 = pd.read_csv('https://github.com/stawiskm/QSAR_Modelbuilding_amesTest/raw/main/data/paperData2.csv')
df2.columns = ['canonical_smiles', 'class']
df3.columns = ['canonical_smiles', 'class']
df2['class'] = df2['class'].replace({'non-mutagens':'Non-toxic'})
df2['class'] = df2['class'].replace({'mutagens':'Toxic'})
df3['class'] = df3['class'].replace({'non-mutagens':'Non-toxic'})
df3['class'] = df3['class'].replace({'mutagens':'Toxic'})

In [7]:
df_all = pd.concat([df1,df2,df3], axis=0)

In [8]:
def cleanSMILES(row):
    cpd = str(row.canonical_smiles).split('.')
    cpd_longest = max(cpd, key = len)
    return cpd_longest

In [9]:
df_all['smiles'] = df_all.apply(cleanSMILES ,axis=1)
df_all = df_all.drop("canonical_smiles",axis=1)
df_all = df_all.rename({"smiles":"canonical_smiles"},axis=1)

df_all=df_all.drop_duplicates(subset=['canonical_smiles'], keep='last')

In [10]:
selection = ["canonical_smiles","class"]
df_all[selection].to_csv('molecule.smi', sep='\t', index=False, header=False)

## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [11]:
! bash padel.sh

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Processing Toxic in molecule.smi (1043/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1044/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1045/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1046/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1047/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1048/6041). Average speed: 0.19 s/mol.
Processing Non-toxic in molecule.smi (1049/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1050/6041). Average speed: 0.19 s/mol.
Processing Non-toxic in molecule.smi (1052/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1051/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1053/6041). Average speed: 0.19 s/mol.
Processing Toxic in molecule.smi (1054/6041). Average speed: 0.19 s/mol.
Processing Non-toxic in molecule.smi (1055/6041).

In [17]:
df = pd.read_csv('descriptors_output.csv')
df = df.rename({'Name': 'class'}, axis='columns')
df.to_csv('QSAR_DescriptorData_ALL.csv', index=False)

## Final Model for deployment

In [13]:
import pickle
import pandas as pd
from lightgbm import LGBMClassifier

In [14]:
df = pd.read_csv("https://raw.githubusercontent.com/stawiskm/QSAR_Modelbuilding_amesTest/main/data/QSAR_DescriptorData_ALL.csv")

In [18]:
X = df.drop(['class'], axis=1)
y = df["class"]

In [19]:
clfmodel = {'boosting_type': 'gbdt', 'learning_rate': 0.126, 'max_depth': 21, 'n_estimators': 96, 'num_leaves': 32, 'subsample_for_bin': 60000}

model =  LGBMClassifier(boosting_type=clfmodel["boosting_type"],
                        learning_rate=clfmodel['learning_rate'],
                        n_estimators=clfmodel['n_estimators'],
                        num_leaves=clfmodel['num_leaves'],
                        subsample_for_bin=clfmodel["subsample_for_bin"],
                        max_depth=clfmodel["max_depth"],
                        random_state=42)

model.fit(X,y)

pickle.dump(model, open('amesTest_model.pkl', 'wb'))

In [20]:
model_reloaded = pickle.load(open("amesTest_model.pkl", 'rb'))
model_reloaded.predict(X)

array(['Non-toxic', 'Non-toxic', 'Non-toxic', ..., 'Non-toxic',
       'Non-toxic', 'Non-toxic'], dtype=object)